In [1]:
pip install numpy

     |████████████████████████████████| 18.2MB 19.2MB/s eta 0:00:01
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.2.5 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement numpy==1.24.1; python_version >= "3.10", but you'll have numpy 1.26.4 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

     |████████████████████████████████| 13.0MB 16.0MB/s eta 0:00:01
     |████████████████████████████████| 512kB 86.8MB/s eta 0:00:01
     |████████████████████████████████| 348kB 79.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4b/d7/ecf66c1cd12dc28b4040b15ab4d17b773b87fa9d29ca16125de01adb36cd/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     |████████████████████████████████| 235kB 86.9MB/s eta 0:00:01
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 4.19.0 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.2.5 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement numpy==1.24.1; python_version >= "3.10", but you'll have numpy 1.26.4 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use

In [1]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Product Dimension Table
categories = ['Dairy', 'Beverages', 'Snacks', 'Cleaning', 'Personal Care', 'Bakery', 'Frozen', 'Produce', 'Meat', 'Cereals']
subcategories = ['Organic', 'Gluten-Free', 'Low-Fat', 'Sugar-Free']
product_df = pd.DataFrame({
    'Product_ID': range(1, 101),
    'Product_Name': ['Product_' + str(i) for i in range(1, 101)],
    'Category': np.random.choice(categories, 100),
    'Subcategory': np.random.choice(subcategories, 100),
})

# Seasonality Dimension Table
date_range = pd.date_range(start='2020-01-01', end='2024-03-31')
seasonality_df = pd.DataFrame({
    'Date': date_range,
    'Day_of_Week': date_range.day_name(),
    'Month': date_range.month_name(),
    'Season': np.where(date_range.month % 12 < 3, 'Winter', 
                       np.where(date_range.month < 6, 'Spring', 
                                np.where(date_range.month < 9, 'Summer', 'Fall'))),
    'Holiday': np.random.choice([True, False], len(date_range), p=[0.1, 0.9]),
})

# Inventory Dimension Table
inventory_items_per_product = 5  # Each product has 5 inventory items
total_inventory_items = len(product_df) * inventory_items_per_product
inventory_dim_df = pd.DataFrame({
    'Inventory_ID': range(1, total_inventory_items + 1),
    'Product_ID': np.repeat(product_df['Product_ID'].values, inventory_items_per_product),
    'Restock_Frequency': np.random.choice(['Weekly', 'Bi-Weekly', 'Monthly'], total_inventory_items),
    'Lead_Time_Days': np.random.randint(1, 15, total_inventory_items),  # Simulated lead time in days for restocking
    #'Perishable': np.random.choice([True, False], total_inventory_items, p=[0.3, 0.7]),
})


<ipython-input-1-5434c3edb9bb>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Base prices for each product within the specified range
base_prices = pd.Series(np.random.uniform(10, 1000, size=len(product_df)), index=product_df['Product_ID'])

# Fact Table: Daily Pricing and Sales Data
daily_data = pd.DataFrame({
    'Date': np.repeat(seasonality_df['Date'].values, len(product_df)),
    'Product_ID': np.tile(product_df['Product_ID'].values, len(seasonality_df)),
})

# Assigning consistent selling prices with minor daily fluctuations
daily_data['Base_Selling_Price'] = daily_data['Product_ID'].map(base_prices)
daily_variation = np.random.normal(1.0, 0.1, size=len(daily_data))  # Small percentage variation each day
daily_data['Selling_Price'] = daily_data['Base_Selling_Price'] * daily_variation

# Define a percentage range to reduce the selling price by, to set the cost price
# For example, cost price will be 40% to 90% of the selling price
cost_price_percentage_min = 0.4
cost_price_percentage_max = 0.9
# Generate a random percentage reduction within the defined range for each product
# This ensures variability but consistency for each product's cost price relative to its selling price
percentage_reduction = pd.Series(np.random.uniform(cost_price_percentage_min, cost_price_percentage_max, size=len(product_df)), index=product_df['Product_ID'])
# Apply the percentage reduction to the selling price to calculate the cost price
# The cost price for each product will now be consistently based on its selling price
daily_data['Cost_Price'] = daily_data.apply(lambda row: row['Selling_Price'] * percentage_reduction[row['Product_ID']], axis=1)

# Competitor prices close but varied around the selling price
daily_data['Competitor_Price'] = daily_data['Selling_Price'] * np.random.uniform(0.9, 1.1, size=len(daily_data))

# Function to adjust units sold based on conditions discussed
def units_sold(row):
    return np.random.randint(100, 5000)  # Ensuring within the range

daily_data['Units_Sold'] = daily_data.apply(units_sold, axis=1)
daily_data['Revenue'] = daily_data['Units_Sold'] * daily_data['Selling_Price']


# Inventory Fact Table: Assuming inventory is adjusted based on sales
inventory_fact_df = pd.DataFrame({
    'Date': np.repeat(seasonality_df['Date'].values, total_inventory_items),
    'Inventory_ID': np.tile(inventory_dim_df['Inventory_ID'].values, len(seasonality_df)),
    'Product_ID': np.tile(inventory_dim_df['Product_ID'].values, len(seasonality_df))
})

# Assuming restocking to initial levels daily for simplicity
inventory_fact_df['Restocked_Units'] = np.where(inventory_fact_df['Date'].dt.dayofweek == 0,
                                                np.random.randint(100, 1000, size=len(inventory_fact_df)),
                                                0)

# Simulate daily sales deductions from inventory
inventory_fact_df['Daily_Sales'] = np.random.randint(10, 100, size=len(inventory_fact_df))

# Calculate net stock level change
inventory_fact_df['Net_Stock_Change'] = inventory_fact_df['Restocked_Units'] - inventory_fact_df['Daily_Sales']



In [3]:
daily_data

,Date,Product_ID,Base_Selling_Price,Selling_Price,Cost_Price,Competitor_Price,Units_Sold,Revenue
0,2020-01-01,1,620.895851,738.331338,523.867455,785.622870,3282,2.423203e+06
1,2020-01-01,2,122.700010,117.030091,54.213319,126.761434,3785,4.429589e+05
2,2020-01-01,3,351.770911,350.153820,149.091977,316.811768,4939,1.729410e+06
3,2020-01-01,4,512.337819,453.589849,391.816273,455.000834,429,1.945900e+05
4,2020-01-01,5,875.480299,875.961161,572.689826,817.710505,1131,9.907121e+05
...,...,...,...,...,...,...,...,...
155195,2024-03-31,96,415.532606,441.877963,257.024851,431.136239,4510,1.992870e+06
155196,2024-03-31,97,318.105320,334.200746,149.171105,307.404064,1073,3.585974e+05
155197,2024-03-31,98,680.349433,672.499505,270.564806,714.116061,1771,1.190997e+06
155198,2024-03-31,99,609.721457,670.126013,284.059349,697.136460,4672,3.130829e+06


In [4]:
# Map day of week to weekend indicator
weekend_mapping = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 1, 'Sunday': 1}
seasonality_df['Is_Weekend'] = seasonality_df['Day_of_Week'].map(weekend_mapping)

# Calculate total units sold for each product
total_units_by_product = daily_data.groupby('Product_ID')['Units_Sold'].sum()
# Identify the top 5 products
top_5_products = total_units_by_product.nlargest(5).index.tolist()

# Update to adjust selling price for holidays and demand fluctuations
def adjust_selling_price(row):
   holiday_factor = 1.1 if row['Is_Holiday'] else 1.0
   weekend_factor = 1.05 if row['Is_Weekend'] else 1.0
   demand_fluctuation = np.random.uniform(0.95, 1.05)
   return row['Base_Selling_Price'] * holiday_factor * weekend_factor * demand_fluctuation
# Update to adjust units sold based on various factors
def adjust_units_sold(row):
   price_sensitivity = -0.1  # Example sensitivity, adjust based on analysis
   holiday_boost = 1.5 if row['Is_Holiday'] else 1.0
   weekend_boost = 1.0
   if row['Product_ID'] in top_5_products and row['Is_Weekend'] and np.random.rand() <0.8:
        weekend_boost = 1.3
   price_factor = 1.0 if row['Selling_Price'] <= row['Competitor_Price'] else 0.5
   price_effect = np.exp(price_sensitivity * (row['Selling_Price'] - row['Competitor_Price']))
   units_sold = row['Units_Sold'] * holiday_boost * weekend_boost * price_factor
   return max(0, np.round(units_sold))  # Ensure units sold is not negative
# Add columns to indicate holidays and weekends
seasonality_df['Is_Holiday'] = seasonality_df['Holiday'].astype(int)

# Map day of week to weekend indicator
weekend_mapping = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 1, 'Sunday': 1}
seasonality_df['Is_Weekend'] = seasonality_df['Day_of_Week'].map(weekend_mapping)

# Merge seasonality info into daily_data
daily_data = daily_data.merge(seasonality_df[['Date', 'Is_Holiday', 'Is_Weekend']], on='Date', how='left')
# Apply adjustments
daily_data['Selling_Price'] = daily_data.apply(adjust_selling_price, axis=1)
daily_data['Units_Sold'] = daily_data.apply(adjust_units_sold, axis=1)
daily_data['Revenue'] = daily_data['Units_Sold'] * daily_data['Selling_Price']
daily_data['Competitor_Price'] = daily_data['Selling_Price'] * np.random.uniform(0.9, 1.1, size=len(daily_data))

# Display a sample of the adjusted daily data
print(daily_data.sample(10))

             Date  Product_ID  Base_Selling_Price  Selling_Price  Cost_Price  \
153963 2024-03-19          64          655.768732     674.018409  378.124216   
5886   2020-02-28          87          151.168203     144.725293   82.833437   
137999 2023-10-11         100          370.947791     353.894784  166.880075   
63636  2021-09-28          37          361.223579     369.599559  186.944529   
72126  2021-12-22          27          689.724014     659.112289  391.571151   
72706  2021-12-28           7          705.236178     728.554192  465.584734   
112164 2023-01-26          65          847.148319     825.294984  449.737667   
115502 2023-03-01           3          351.770911     366.336222  136.798339   
62838  2021-09-20          39          671.886920     667.260163  223.703945   
38637  2021-01-21          38          942.308455     950.066047  684.481898   

        Competitor_Price  Units_Sold       Revenue  Is_Holiday  Is_Weekend  
153963        703.030060      1989.0  1.34

In [5]:
num_inventory_items={key:5 for key in range(1,101)}


In [6]:
# First, ensure that the 'Units_Sold' from daily_data is merged into inventory_fact_df
inventory_fact_df = inventory_fact_df.merge(daily_data, on=['Date', 'Product_ID'], how='left')

# Initialize an empty list to hold the distributed sales data
distributed_sales = []

# Iterate over each group of items for the same product and date in the inventory fact DataFrame
for (date, product_id), group in inventory_fact_df.groupby(['Date', 'Product_ID']):
    items = num_inventory_items[product_id]
    total_sales = group['Units_Sold'].iloc[0]  # Total sales for this product on this date
    each_item_sales = total_sales // items  # Sales per inventory item
    remainder = total_sales % items  # Remainder to be distributed

    # Distribute sales across inventory items, adding the remainder to the last item
    sales_distribution = [each_item_sales] * items
    sales_distribution[-1] += remainder

    # Extend the distributed_sales list with the calculated sales distribution
    distributed_sales.extend(sales_distribution)

# Assign the distributed sales to the 'Daily_Sales' column
inventory_fact_df['Daily_Sales'] = distributed_sales

# Drop the now unnecessary 'Units_Sold' column
inventory_fact_df.drop(columns=['Units_Sold'], inplace=True)

# Display the updated inventory fact DataFrame
inventory_fact_df.head()


,Date,Inventory_ID,Product_ID,Restocked_Units,Daily_Sales,Net_Stock_Change,Base_Selling_Price,Selling_Price,Cost_Price,Competitor_Price,Revenue,Is_Holiday,Is_Weekend
0,2020-01-01,1,1,0,656.0,-50,620.895851,642.499419,523.867455,584.063074,2.108683e+06,0,0
1,2020-01-01,2,1,0,656.0,-88,620.895851,642.499419,523.867455,584.063074,2.108683e+06,0,0
2,2020-01-01,3,1,0,656.0,-16,620.895851,642.499419,523.867455,584.063074,2.108683e+06,0,0
3,2020-01-01,4,1,0,656.0,-56,620.895851,642.499419,523.867455,584.063074,2.108683e+06,0,0
4,2020-01-01,5,1,0,658.0,-56,620.895851,642.499419,523.867455,584.063074,2.108683e+06,0,0


In [ ]:
def calculate_inventory(df, max_stock_per_product):
   df = df.sort_values(by='Date').reset_index(drop=True)  # Ensure chronological order and reset index
   # Initialize columns
   df['Restocked_Units'] = 0
   df['Net_Stock_Change'] = -df['Daily_Sales']  # Assuming sales reduce stock
   df['Running_Stock'] = 0
   for i in range(len(df)):
       if i == 0:  # First entry for the product's inventory items
           # Set initial stock levels based on max_stock_per_product
           df.at[i, 'Running_Stock'] = max_stock_per_product
       else:
           # Calculate restocked units based on sales and max stock constraint
           restock_needed = min(df.at[i, 'Daily_Sales'], max_stock_per_product - df.at[i-1, 'Running_Stock'])
           df.at[i, 'Restocked_Units'] = restock_needed
           # Update Net_Stock_Change to reflect restocking
           df.at[i, 'Net_Stock_Change'] = restock_needed - df.at[i, 'Daily_Sales']
           # Update Running_Stock
           df.at[i, 'Running_Stock'] = df.at[i-1, 'Running_Stock'] + df.at[i, 'Net_Stock_Change']
           # Ensure Running_Stock is within [0, max_stock_per_product]
           df.at[i, 'Running_Stock'] = max(min(df.at[i, 'Running_Stock'], max_stock_per_product), 0)
   return df

inventory_fact_df = inventory_fact_df.groupby('Product_ID').apply(calculate_inventory, max_stock_per_product=2000).reset_index(drop=True)

In [ ]:
# Display sample data
print(product_df.head())
print(seasonality_df.head())
print(inventory_dim_df.head())
print(daily_data.head())
print(inventory_fact_df.head())


In [ ]:
# Seed for reproducibility
np.random.seed(42)

# Campaign details
n_campaigns = 100
campaign_types = ['Email Blast', 'Social Media Ad', 'Holiday Sale', 'Limited Time Offer', 'New Product Launch']
discount_types = ['Percentage Off', 'Buy One Get One Free', 'Fixed Amount Off']
product_categories = ['Dairy', 'Beverages', 'Snacks', 'Cleaning', 'Personal Care', 'Bakery', 'Frozen', 'Produce', 'Meat', 'Cereals']

# Campaign DataFrame
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2024-12-31')
date_range = (end_date - start_date).days

campaign_df = pd.DataFrame({
    'Campaign_ID': range(1, n_campaigns + 1),
    'Campaign_Type': np.random.choice(campaign_types, n_campaigns),
    'Discount_Type': np.random.choice(discount_types, n_campaigns),
    'Targeted_Category': np.random.choice(product_categories, n_campaigns),
    'Start_Date': start_date + pd.to_timedelta(np.random.randint(0, date_range - 30, size=n_campaigns), unit='D'),
})


# Assign End_Date, ensuring it's after Start_Date
campaign_df['End_Date'] = campaign_df['Start_Date'] + pd.to_timedelta(np.random.randint(1, 30, size=n_campaigns), unit='D')
campaign_df['End_Date'] = campaign_df.apply(lambda row: row['Start_Date'] + pd.DateOffset(days=1) if row['End_Date'] < row['Start_Date'] else row['End_Date'], axis=1)

print(campaign_df)
campaign_df.to_csv('campaign_data.csv')


In [ ]:
daily_data.to_csv('daily_sales_data.csv')
campaign_df.to_csv('campaign_data.csv')
inventory_dim_df.to_csv('inventory_dim.csv')
inventory_fact_df.to_csv('inventory_fact.csv')
product_df.to_csv('product_dim.csv')
seasonality_df.to_csv('seasonality_dim.csv')